### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data.sort_values('school_name', inplace=True)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
schools = school_data_complete['school_name'].unique()
schoolCount = len(schools)
studentCount = school_data_complete['Student ID'].count()
districtSummaryDF = pd.DataFrame({'Total Schools' : schoolCount, 
                                  'Total Students' : studentCount, 
                                  'Total Budget' : [school_data['budget'].sum()],
                                  'Avg Math Score' : [school_data_complete['math_score'].mean()],
                                  'Avg Reading Score' : [school_data_complete['reading_score'].mean()] })

#get the score above 70 and calculate percentage passing
studentsPassingMath = school_data_complete['math_score'] >=70
percentPassingMath = studentsPassingMath.sum()/studentCount *100
studentsPassingRead = school_data_complete['reading_score'] >=70
percentPassingRead = studentsPassingRead.sum()/studentCount *100
studentsPassingBoth = (school_data_complete['math_score'] >=70) & (school_data_complete['reading_score'] >=70)
percentPassingBoth = studentsPassingBoth.sum()/studentCount *100

districtSummaryDF['% Passing Math'] = percentPassingMath
districtSummaryDF['% Passing Reading'] = percentPassingRead
districtSummaryDF['% Overall Passing'] = percentPassingBoth

#add formatting to the table
districtSummaryDF[['% Passing Math', '% Passing Reading', '% Overall Passing', 'Avg Math Score', 'Avg Reading Score']] = districtSummaryDF[['% Passing Math', '% Passing Reading', '% Overall Passing', 'Avg Math Score', 'Avg Reading Score']].applymap("{:,.2f}".format)
districtSummaryDF['Total Budget'] = districtSummaryDF['Total Budget'].map("${:,.2f}".format)

districtSummaryDF

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
summary = school_data_complete.groupby('school_name').agg(School_Type = ('type', 'first'),
                                                    Total_Students = ('Student ID', 'count'), 
                                                    Total_School_Budget = ('budget', 'first'),
                                                    Avg_Math_Score= ('math_score', np.mean),
                                                    Avg_Reading_Score = ('reading_score', np.mean))

#Collect the number of students with passing score
PassedStudentCountForMath = school_data_complete.loc[school_data_complete['math_score'] >=70].groupby('school_name').agg(StudentCount = ('math_score', 'count'))
PassedStudentCountForReading = school_data_complete.loc[school_data_complete['reading_score'] >=70].groupby('school_name').agg(StudentCount = ('reading_score', 'count'))
PassedStudentCountForBoth = school_data_complete.loc[(school_data_complete['reading_score'] >=70) & (school_data_complete['math_score'] >=70)].groupby('school_name').agg(StudentCount = ('reading_score', 'count'))

#calculate the passing percentage of students and add the columns
summary['% Passing Math'] = PassedStudentCountForMath['StudentCount']/summary['Total_Students']*100
summary['% Passing Reading'] = PassedStudentCountForReading['StudentCount']/summary['Total_Students']*100
summary['% Overall Passing'] = PassedStudentCountForBoth['StudentCount']/summary['Total_Students']*100
summary['Per Student Budget'] = summary['Total_School_Budget']/summary['Total_Students']

#rename, reorder, format columns
StudentBudgetCol = summary['Per Student Budget']
summary.drop(labels=['Per Student Budget'], axis=1, inplace = True)
summary.insert(3, 'Per Student Budget', StudentBudgetCol)
summary = summary.rename(columns={'School_Type' : 'School Type', 'Total_Students':'Total Students', 'Total_School_Budget' : 'Total School Budget',
                                 'Avg_Math_Score' : 'Avg Math Score', 'Avg_Reading_Score' : 'Avg Reading Score'})
displaySummary = summary.copy()
displaySummary[['Avg Math Score','Avg Reading Score','% Passing Math','% Passing Reading','% Overall Passing']] = displaySummary[['Avg Math Score','Avg Reading Score','% Passing Math','% Passing Reading','% Overall Passing']].applymap("{:,.2f}".format)
displaySummary[['Total School Budget', 'Per Student Budget']] = displaySummary[['Total School Budget','Per Student Budget']].applymap("${:,.2f}".format)
displaySummary.index.name = ''

#display results
displaySummary

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
summary.nlargest(5, '% Overall Passing')


,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
summary.nsmallest(5, '% Overall Passing')

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
gradesTable = school_data_complete.pivot_table(values=['math_score'], index='school_name', columns='grade',
                                     aggfunc={'math_score' : np.mean})

col = gradesTable[('math_score', '9th')]
gradesTable.drop(labels=[('math_score', '9th')], axis=1, inplace = True)
gradesTable.insert(0, ('math_score', '9th'), col)
gradesTable.columns = ['9th', '10th', '11th', '12th']
gradesTable.index.name = ''
gradesTable[['9th', '10th', '11th', '12th']] = gradesTable[['9th', '10th', '11th', '12th']].applymap("{:,.2f}".format)
gradesTable

,9th,10th,11th,12th
,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
gradesTable = school_data_complete.pivot_table(values=['reading_score'], index='school_name', columns='grade',
                                     aggfunc={'reading_score' : np.mean})

col = gradesTable[('reading_score', '9th')]
gradesTable.drop(labels=[('reading_score', '9th')], axis=1, inplace = True)
gradesTable.insert(0, ('reading_score', '9th'), col)
gradesTable.columns = ['9th', '10th', '11th', '12th']
gradesTable.index.name = ''
gradesTable[['9th', '10th', '11th', '12th']] = gradesTable[['9th', '10th', '11th', '12th']].applymap("{:,.2f}".format)
gradesTable

,9th,10th,11th,12th
,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
newsummary = summary.copy()
newsummary['Spending Ranges(Per Student)'] = pd.cut(newsummary['Per Student Budget'], bins=[0,585,630,645,680], labels=['<$585','$585-630', '$630-645','$645-680'])
spendingGoups = newsummary.groupby('Spending Ranges(Per Student)').agg(Avg_Math_Score= ('Avg Math Score', np.mean),
                                                    Avg_Reading_Score = ('Avg Reading Score', np.mean),
                                                           Passing_Math=('% Passing Math', np.mean),
                                                           Passing_Reading =('% Passing Reading', np.mean),
                                                           Overall_Passing =('% Overall Passing', np.mean))
spendingGoups.columns=['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
spendingGoups[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = spendingGoups[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']].applymap("{:,.2f}".format)
spendingGoups

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges(Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
schoolsizeSummary = summary.copy()
schoolsizeSummary['Size'] = school_data['size'].values
schoolsizeSummary['School Size'] = pd.cut(schoolsizeSummary['Size'], bins=[1,1000,2000,5000], labels=['Small(<1000)', 'Medium(1000-2000)','Large(2000-5000)'])
sizeGroups = schoolsizeSummary.groupby('School Size').agg(Avg_Math_Score= ('Avg Math Score', np.mean),
                                                    Avg_Reading_Score = ('Avg Reading Score', np.mean),
                                                           Passing_Math=('% Passing Math', np.mean),
                                                           Passing_Reading =('% Passing Reading', np.mean),
                                                           Overall_Passing =('% Overall Passing', np.mean))
sizeGroups.columns=['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
sizeGroups[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = sizeGroups[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']].applymap("{:,.2f}".format)

sizeGroups

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.88
Medium(1000-2000),83.37,83.86,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
schType = summary.groupby('School Type').agg(Avg_Math_Score= ('Avg Math Score', np.mean),
                                                    Avg_Reading_Score = ('Avg Reading Score', np.mean),
                                                           Passing_Math=('% Passing Math', np.mean),
                                                           Passing_Reading =('% Passing Reading', np.mean),
                                                           Overall_Passing =('% Overall Passing', np.mean))
schType.columns=['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
schType[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = schType[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']].applymap("{:,.2f}".format)

schType

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
